In [11]:
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
import datetime
from datetime import datetime, timedelta

In [13]:
import os
from subprocess import check_output
import warnings
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from datetime import datetime as dt

In [14]:
import dash
from dash import Dash,dcc,html,Input,Output
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pathlib

In [15]:
import plotly as pl
import plotly.express as px

In [16]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [17]:
import datetime

In [18]:
stocks=["PG","KO","UL","MMM","JNJ","CAT"]

start_date="2020-01-01"
end_date=datetime.datetime.now().strftime("%Y-%m-%d")

data=yf.download(stocks,start=start_date,end=end_date)["Adj Close"]
returns=data.pct_change().dropna()
data

[*********************100%***********************]  6 of 6 completed


Ticker,CAT,JNJ,KO,MMM,PG,UL
Date,,,,,,
2020-01-02 00:00:00+00:00,134.881516,128.083755,47.451931,123.614929,109.081001,47.854389
2020-01-03 00:00:00+00:00,133.008759,126.600815,47.193058,122.550484,108.347374,47.787247
2020-01-06 00:00:00+00:00,132.919159,126.442863,47.175793,122.667236,108.497635,47.837601
2020-01-07 00:00:00+00:00,131.162918,127.215027,46.813374,122.172768,107.825867,47.065479
2020-01-08 00:00:00+00:00,132.327759,127.197502,46.899662,124.047585,108.285500,46.864052
...,...,...,...,...,...,...
2024-11-06 00:00:00+00:00,416.880005,157.880005,63.700001,133.919998,161.050003,58.796715
2024-11-07 00:00:00+00:00,408.209991,156.729996,63.660000,133.070007,163.410004,59.134003
2024-11-08 00:00:00+00:00,393.369995,155.470001,63.919998,134.339996,167.710007,59.000000


In [19]:
app=Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard"),
    dcc.Dropdown(
        id="ticker-dropdown",
        options=[{"label":ticker,"value":ticker} for ticker in stocks],
        value="PG",
        multi=True
    ),
    dcc.RadioItems(
        id="metric-radio",
        options=[
            {"label":"Precios de cierre","value":"precios"},
            {"label":"Retornos","value":"retornos"}
        ],
        value="precios"
    ),
    dcc.DatePickerRange(
        id="date-picker",
        start_date=start_date,
        end_date=end_date
    ),
    dcc.Graph(id="price-chart"),
    dcc.Graph(id="return-chart")
])

@app.callback(
    [Output("price-chart","figure"),
     Output("return-chart","figure")],
    [Input("ticker-dropdown","value"),
     Input("metric-radio","value"),
     Input("date-picker","start_date"),
     Input("date-picker","end_date")]
)
def update_charts(selected_tickers,selected_metric,fechainicio,fechafin):
    fechainicio=pd.to_datetime(fechainicio).tz_localize("UTC")
    fechafin=pd.to_datetime(fechafin).tz_localize("UTC")

    filtered_data=data.loc[fechainicio:fechafin,selected_tickers]
    filtered_returns=returns.loc[fechainicio:fechafin,selected_tickers]

    if selected_metric == "precios":
        fig=px.line(filtered_data,title="Precios de cierre")
        fig.update_layout(xaxis_title="Fecha",yaxis_title="Precio ($)")
    else:
        fig=px.line(filtered_returns,title="Retornos diarios")
        fig.update_layout(xaxis_title="Fecha",yaxis_title="Retorno (%)")

    dist_fig=make_subplots(rows=1,cols=1)
    for ticker in selected_tickers:
        dist_fig.add_trace(go.Histogram(x=filtered_returns[ticker],name=ticker,opacity=0.6))
    dist_fig.update_layout(title="Distribución de retornos", barmode="overlay")
    dist_fig.update_traces(opacity=0.75)

    return fig,dist_fig

if __name__ == "__main__":
    app.run_server(debug=True)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File /opt/anaconda3/lib/python3.12/site-packages/pandas/core/indexes/datetimes.py:603, in DatetimeIndex.get_loc(
    self=DatetimeIndex(['2020-01-03 00:00:00+00:00', '202...4[ns, UTC]', name='Date', length=1224, freq=None),
    key='P'
)
    602 try:
--> 603     parsed, reso = self._parse_with_reso(key)
        key = 'P'
        self = DatetimeIndex(['2020-01-03 00:00:00+00:00', '2020-01-06 00:00:00+00:00',
               '2020-01-07 00:00:00+00:00', '2020-01-08 00:00:00+00:00',
               '2020-01-09 00:00:00+00:00', '2020-01-10 00:00:00+00:00',
               '2020-01-13 00:00:00+00:00', '2020-01-14 00:00:00+00:00',
               '2020-01-15 00:00:00+00:00', '2020-01-16 00:00:00+00:00',
               ...
               '2024-10-30 00:00:00+00:00', '2024-10-31 00:00:00+00:00',
               '2024-11-01 00:00:00+00:00', '2024-11-